Welcome to the Aiven Clickhouse challenge!

With the launch of Aiven for Clickhouse We finally have the "C" for our KFC stack, the best combination of tools for realtime aleart and historic analytics.

The stack untilises Apache Kafka, Apache Flink, and Clickhouse to build up a robust, scleable architecture for getting the most from your data, whether it be batched ETL or realtime sensors.

For this challenge, We'll be using realtime data from [digitransit](https://digitransit.fi/en/developers/apis/4-realtime-api/) to plot maps and find new insights.

Digitransit provides reatime bus informantion and positioning information for busses in Helsinki, We've saved out a few weeks worth of data as a CSV to experiemnt with.

The goal is to find the best bus routes between the different KFC restaurants in Helsinki. plot them on a map and tag us / DM us on twitter with a screen shot of a map with these routes on it. 

The prize for this is a Limited edition Hat + The knowledge that you can indeed navigate public transport with the right SQL statement.

This notbook domes with a few starter queries and a mapping library to get you started.

For more help/ questions ping us on twitter [@aiven_io](https://twitter.com/aiven_io), or me [@Bengamble7](https://twitter.com/BenGamble7) and we'll provide hints and even more terrible puns and clickhouse, or any of our 11 ~~secrect herbs and spices~~ producst



In [20]:

from clickhouse_driver import Client
client = Client('YOUR-CLICKHOUSE-URL',
                user='avnadmin',
                password='YOUR-PASSWORD',
                secure=True,
                port='YOUR-PORT',
                )

#to confirm its all wowrking
client.execute('SHOW DATABASES')

[('Digitransit',),
 ('INFORMATION_SCHEMA',),
 ('default',),
 ('information_schema',),
 ('kafka',),
 ('service_digitransit-kafka',),
 ('system',)]

In [7]:
first_query = 'select lat, long from default.DigitransitINPUT where (veh = 1344) AND (spd > 15.0) limit 500'

out = client.execute(first_query)

len(out)


93

In [17]:
count_query = 'select count() from default.DigitransitINPUT'

out = client.execute(count_query)

out

[(15992100,)]

In [11]:
next_query = 'select lat, long from default.DigitransitINPUT where  (spd > 45.0) limit 500'

out = client.execute(next_query)

len(out)


199

In [12]:
import folium
m = folium.Map(location=[60.40, 25.1067], zoom_start=11)

for point in out:
    try:
        folium.Marker(point).add_to(m)
    except:
        pass
    
m

In [67]:
marker_colors = ['red', 'darkred',  'lightred', 'orange', 'beige',
                 'green', 'darkgreen', 'lightgreen',
                 'blue', 'darkblue', 'cadetblue', 'lightblue',
                 'purple',  'darkpurple', 'pink',
                 'white', 'gray', 'lightgray', 'black']


                 

In [18]:
out = client.execute('select max (spd) from default.DigitransitINPUT')
out


Error on digitransit-clickhouse-dev-sandbox.aivencloud.com:12691 ping: [WinError 10054] An existing connection was forcibly closed by the remote host
Connection was closed, reconnecting.


[(48.09,)]

This is an interesting one as it lets us map colors to speed :)


In [62]:
import math
def colorise(speed_in: float)-> str:
    return marker_colors[math.floor(speed_in/50)]

In [46]:
def make_map(my_points):
    map = folium.Map(location=[60.40, 25.1067], zoom_start=11)

    for point in my_points:
        try:
            folium.Marker(location=[point[0], point[1]],  icon=folium.Icon(color=colorise(point[2])) ).add_to(map)
        except:
            pass

    return map

In [69]:
next_query = 'select lat, long, spd from default.DigitransitINPUT limit 500'
out = client.execute(next_query)

map = folium.Map(location=[60.40, 25.1067], zoom_start=11)
color_str =''
for point in out:
    try:
        folium.Marker(location=[point[0], point[1]], color = colorise(point[2])).add_to(map)

    except:
        pass

        
map